In [37]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score


In [38]:
def process_csv_feautures(file_paths):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([row['instructions'],row['br_pred'],row['br_mis_pred'],row['cpu-cycles'],row['vfp_spec'],row['st_spec'],row['ld_spec'],row['l1d_cache_wr'],row['l1d_cache_rd'],row['l1d_cache_rd'],row['l1d_cache'],row['l1d_tlb_refill_wr'],row['l1d_tlb_refill_rd'],row['l1d_tlb'],row['l1i_tlb'],row['l1i_cache_refill'],row['l1i_cache'],row['l1i_tlb_refill'],row['bus_access_rd'],row['bus_access'],row['l2d_cache_wr'],row['l2d_cache_rd'],row['l2d_cache'],row['mem_access_wr'],row['mem_access_rd'],row['mem_access'],ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

in_order_files = [
    'inst_aligned_humerus_cpu0_1.5GHz_500.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_502.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_505.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_520.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_523.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_525.csv',
#    'inst_aligned_humerus_cpu0_1.5GHz_531.csv'
#    'inst_aligned_humerus_cpu0_1.5GHz_541.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_548.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_557.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_503.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_507.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_508.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_510.csv',
#    'inst_aligned_humerus_cpu0_1.5GHz_511.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_519.csv',
#    'inst_aligned_humerus_cpu0_1.5GHz_521.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_526.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_527.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_538.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_544.csv',
    'inst_aligned_humerus_cpu0_1.5GHz_549.csv'
]
    
out_of_order_files = [
    'inst_aligned_humerus_cpu0_3.0GHz_500.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_502.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_505.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_520.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_523.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_525.csv',
#    'inst_aligned_humerus_cpu0_3.0GHz_531.csv'
#    'inst_aligned_humerus_cpu0_3.0GHz_541.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_548.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_557.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_503.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_507.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_508.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_510.csv',
#    'inst_aligned_humerus_cpu0_3.0GHz_511.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_519.csv',
#    'inst_aligned_humerus_cpu0_3.0GHz_521.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_526.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_527.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_538.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_544.csv',
    'inst_aligned_humerus_cpu0_3.0GHz_549.csv'
]

in_order_hardware_counters_features = process_csv_feautures(in_order_files)
out_of_order_ipc = process_csv_ipc(out_of_order_files)


# 
data = {
    'In Order': in_order_hardware_counters_features,
    'Out of Order': out_of_order_ipc
}

length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))

# Shorten the longen array
in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
out_of_order_ipc = out_of_order_ipc[:length]

# separate features and target
X = pd.DataFrame(in_order_hardware_counters_features, columns=['instructions','br_pred','br_mis_pred','cpu-cycles','vfp_spec','st_spec','ld_spec','l1d_cache_wr','l1d_cache_rd','l1d_cache_rd','l1d_cache','l1d_tlb_refill_wr','l1d_tlb_refill_rd','l1d_tlb','l1i_tlb','l1i_cache_refill','l1i_cache','l1i_tlb_refill','bus_access_rd','bus_access','l2d_cache_wr','l2d_cache_rd','l2d_cache','mem_access_wr','mem_access_rd','mem_access','ipc'])  # Features
y = pd.Series(out_of_order_ipc) 

# split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100)
gb_model = GradientBoostingRegressor(n_estimators=100)
ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
ensemble_model.fit(X_train, y_train)
ensemble_predictions = ensemble_model.predict(X_test)
# Calculate R²
ensemble_r2 = r2_score(y_test, ensemble_predictions)


#bruh wut
train_predictions = ensemble_model.predict(X_train)
test_predictions = ensemble_model.predict(X_test)

# Calculate R² for training and testing sets
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"Training R²: {train_r2}")
print(f"Testing R²: {test_r2}")

# Create a DataFrame for Actual and Predicted values
results_df = pd.DataFrame({
    'In Order IPC': X_test['ipc'],
#    'L2D Cache Refil Value': X_test['armv8_pmuv3/l2d_cache_refil'],
#    'L1D Cache Refil Value': X_test['armv8_pmuv3/l1d_cache_refil'],
#    'L1I Cache Refil Value': X_test['armv8_pmuv3/l1i_cache_refil'],
#    'Inst Retire': X_test['armv8_pmuv3/inst_retire'],
#    'Branch Retire': X_test['armv8_pmuv3/br_retire'],
    'Out of Order IPC (Actual)': y_test,
    'Out of Order IPC (Predicted)': ensemble_predictions
})

results_df.to_csv('predicted_vs_actual_aligned.csv', index=False)
print("Predicted vs Actual values have been written to 'predicted_vs_actual_aligned.csv'.")


Training R²: 0.9977186399432171
Testing R²: 0.9862637261209638
Predicted vs Actual values have been written to 'predicted_vs_actual_aligned.csv'.
